### AlimConfiance Groupe 4 : Audrey, Jaufret, Babacar

## Extract_Transformation_AlimConfiance

1. Importation des librairies et du dataset

In [27]:
#Importation des librairies
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from IPython.display import display
from sklearn import preprocessing

# Importation du dataset depuis csv
export_alim_path='data/raw/export_alimconfiance.csv'
df=pd.read_csv(export_alim_path, sep=";")

# Affichage des données
df.head()

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite
0,NATURALIA FRANCE (NATURALIA FRANCE),"3,02475E+13",45 AV PIERRE LAROUSSE,92240,MALAKOFF,15914760,2021-03-03T01:00:00+01:00,Alimentation générale,Satisfaisant,NaN,"48.822231,2.303769",Alimentation générale,Autres
1,KERMOYAN (RESTAU RAPIDE),"3,0112E+13",MARCHE,69500,BRON,16252422,2021-11-26T01:00:00+01:00,Restaurant,Très satisfaisant,NaN,"45.743534,4.922893",Restaurant,Autres
2,NATURALIA (NATURALIA FRANCE),"3,02475E+13",36 RUE EUGENE EICHENBERGER,92800,PUTEAUX,15903092,2021-02-22T01:00:00+01:00,Alimentation générale,Très satisfaisant,NaN,"48.881317,2.23746",Alimentation générale,Autres
3,JARDINERIE DE CHEVREUSE (JARDINERIE DE CHEVREUSE),"3,08907E+13",LE BREUIL,78460,CHEVREUSE,15901554,2021-02-19T01:00:00+01:00,_,Satisfaisant,NaN,"48.698235,2.027014",NaN,Autres
4,INTERMARCHE (VIRODIS),"3,06689E+13",38 AV LE FOLL,94290,VILLENEUVE LE ROI,15968279,2021-04-14T02:00:00+02:00,Alimentation générale,Satisfaisant,NaN,"48.730573,2.426559",Alimentation générale,Autres


2. Affichage des informations descriptives relatives aux variables

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35662 entries, 0 to 35661
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   APP_Libelle_etablissement           35662 non-null  object
 1   SIRET                               35662 non-null  object
 2   Adresse_2_UA                        35211 non-null  object
 3   Code_postal                         35662 non-null  object
 4   Libelle_commune                     35662 non-null  object
 5   Numero_inspection                   35662 non-null  object
 6   Date_inspection                     35662 non-null  object
 7   APP_Libelle_activite_etablissement  35662 non-null  object
 8   Synthese_eval_sanit                 35662 non-null  object
 9   Agrement                            9840 non-null   object
 10  geores                              34563 non-null  object
 11  filtre                              25999 non-null  ob

3. Supression des colonnes jugées non pertinantes

In [29]:
# Suppression des colonnes
df. drop(['APP_Libelle_etablissement', 'Adresse_2_UA', 'Libelle_commune', 'Numero_inspection','Date_inspection','geores', 'filtre'], axis = 1, inplace = True)
#Affichage des colonnes restantes
#On peut remarquer que toutes les valeurs sont non-null mise à part dans la colonne Agrement
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35662 entries, 0 to 35661
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   SIRET                               35662 non-null  object
 1   Code_postal                         35662 non-null  object
 2   APP_Libelle_activite_etablissement  35662 non-null  object
 3   Synthese_eval_sanit                 35662 non-null  object
 4   Agrement                            9840 non-null   object
 5   ods_type_activite                   35662 non-null  object
dtypes: object(6)
memory usage: 1.6+ MB


4. Conversion des valeurs

In [30]:
# SIRET
df.drop(df[df['SIRET'].str.contains("^[a-z].*$", regex=True, case=False)].index, inplace=True)
df['SIRET']=df['SIRET'].astype('int64')

# Code_postal
# Récupération du département dans le code postal (suppression des 3 dernier chiffres)
df['Code_postal'] = df['Code_postal'].apply(lambda x: str(x)[:2])
df['Code_postal'] = df['Code_postal'].astype(int)

# Agrement
# Remplace les valeurs abs par des 0 dans la colonne
df['Agrement'] = df['Agrement'].fillna(0)
# Change le type de données en string (passage obligatoire car présence de lettre dans les agréments)
df['Agrement']=df['Agrement'].astype(str)
# Remplace les valeurs autre que 0 par 1 et converti la colonne en int
df['Agrement'] = (df['Agrement'] != 0).astype(int)

# Synthese_eval_sanit
#Transformation de la colonne Synthese_eval_sanit grâce à get_dummies
# Répartition des différents niveaux d'hygiene dans 4 colonnes
df = pd.get_dummies(df, columns=['Synthese_eval_sanit'])


# Mise en forme de la colonne libelle_activite
# Suppression des lignes avec libelles manquants
df.drop(df[df['APP_Libelle_activite_etablissement'].str.contains("_")==True].index, inplace=True)

# sépérations des libelles d'activités
df_libelle = df['APP_Libelle_activite_etablissement']
delimiter = '|'
df_length = len(df_libelle)
s = []
for i in range(0, df_length):
    row = df_libelle.iloc[i]
    lib_activite = row.split(delimiter)
    s = s + lib_activite
d = {'APP_Libelle_activite_etablissement': s}
df_libelle = pd.DataFrame(data=d)
df_libelle.drop_duplicates(subset=['APP_Libelle_activite_etablissement'], keep='first', ignore_index=True ,inplace=True)

#Application d'un labelEncoder sur le df_libelle 
le = preprocessing.LabelEncoder()

df_libelle.astype(str)
df_libelle_fit=le.fit_transform(df_libelle)
df_libelle_fit=pd.DataFrame(df_libelle_fit, columns=['LabelEncoder'])
df_libelle=pd.concat([df_libelle,df_libelle_fit], axis=1)

#Mise en forme de la colonne ods_type_activite
# df_type_activite=(df['ods_type_activite'].astype(str)).drop_duplicates()
# df_type_activite_fit=le.fit_transform(df_type_activite)
# df_type_activite_fit=pd.DataFrame(df_type_activite_fit, columns=['LabelEncoder'])
# df_type_activite=pd.concat([df_type_activite,df_type_activite_fit], axis=1)
# print(df_type_activite.head())

# Verifications
df.info()


ValueError: invalid literal for int() with base 10: '3,02475E+13'

In [ ]:
#SAVE_CSV_FILEPATH =(r"C:\Users\diaba\Documents\SimplonIAMicrosoft_21-23\AlimConfiance\AlimConfiance_Clean.csv")
SAVE_CSV_FILEPATH =("modele/data/clean/AlimConfiance_Clean.csv")
df.to_csv(path_or_buf=SAVE_CSV_FILEPATH, sep=';', index=False)
